In [ ]:
### Mount google drive if available
try:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/term_paper/'
    in_colab = True
except:
    drive_path = ''
    in_colab = False

In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# smplx
need_smplx=False
try:
    import smplx
except ModuleNotFoundError:
    need_smplx=True

if need_smplx:
    !pip install smplx
    !git clone https://github.com/vchoutas/smplx
    %cd smplx
    !python setup.py install
    %cd ..


# detectron2
need_detectron=False
try:
    import detectron2
except ModuleNotFoundError:
    need_detectron=True   

if need_detectron:
    !git clone https://github.com/facebookresearch/detectron2.git detectron2_repo
    !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
### CAREFUL: Using the --subdivision flag disables saving displacement map as a texture
if in_colab:
    %run main.py --subjects '[1, 70, 122]' --gdrive {drive_path} --saveobj --smoothing
else:
    %run main.py --subjects '[1, 70, 122]' --saveobj --smoothing

In [ ]:
import smplx
import numpy as np
import matplotlib.pyplot as plt
from utils.smplx_to_disps import smplx2disps
from utils.inpainting import get_disps_inpaint

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

subjects = range(1,618)

obj_path = drive_path + 'smplx/smplx_uv.obj'
uv_mask_img = drive_path + 'smplx/smplx_uv.png'

save_path_imgs = drive_path + 'humbi_output/humbi_smplx_disps'
save_path_npz = drive_path + 'humbi_output/humbi_smplx_npz'

smplx_model_path = drive_path + 'smplx'
smplx_model = smplx.SMPLXLayer(smplx_model_path, 'neutral').to(device)

for subject in subjects:
    geometry_filename = os.path.join(save_path_npz, 'output_subject_%d.npz' % subject)

    if os.path.exists(geometry_filename):
        npzfile = np.load(geometry_filename)

        body_pose = torch.Tensor(npzfile['body_pose']).to(device)
        betas = torch.Tensor(npzfile['betas']).to(device)
        scale = torch.Tensor(npzfile['scale']).to(device)
        verts_disps = torch.Tensor(npzfile['verts_disps']).to(device)

        learned_geometry = smplx2disps(smplx_model, betas, scale, verts_disps, False, smoothing=1)[0]

        disps_x = get_disps_inpaint(subject, learned_geometry[:,0], obj_path, uv_mask_img, mask_disps=True)[0]
        disps_y = get_disps_inpaint(subject, learned_geometry[:,1], obj_path, uv_mask_img, mask_disps=True)[0]
        disps_z = get_disps_inpaint(subject, learned_geometry[:,2], obj_path, uv_mask_img, mask_disps=True)[0]
        displacement_map = torch.cat((disps_x.unsqueeze(2), disps_y.unsqueeze(2), disps_z.unsqueeze(2)), dim=2)

        os.makedirs(save_path_imgs, exist_ok=True)
        disp_filename = os.path.join(save_path_imgs, 'disp_texture_%d.tiff' % subject)
        plt.imsave(disp_filename, displacement_map.cpu().numpy() + 0.5)

        # TODO: replace current npzfile with corrected learned_geometry